In [12]:
#文档信息向量化就是将信息数值化，从而便于建模分析
#词袋模型
#1.词袋模型的gensim实现
from gensim.corpora import Dictionary
from MyDataFrame import MyDataFrame

mdf = MyDataFrame()
df = mdf.new_DataFrame()
df2 = mdf.m_cut(df)
text = [df2['fenci'][0]]#得到第一个分词后的list，并放入另一个list中
dct = Dictionary(text)#形成字典

print(dct.num_pos) #处理多少个词，1142
print(dct.num_docs)#处理几个文档，1
print(dct.num_nnz)#477，最后字典中保留这些个词

dct.token2id#显示字典
#-------------------已形成字典（这里仅一个文件）-----------------

#2.转为bow稀疏向量
dct.doc2bow(["人才","信息","系统","优化","处理"],return_missing = True)#在已有字典中查询给出的词条是有出现
#显示：([(100, 1), (103, 1), (112, 1), (371, 1)], {'处理': 1})



['0', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '4', '5', '6', '7', '8', '9']
1142
1
477


([(100, 1), (103, 1), (112, 1), (371, 1)], {'处理': 1})

In [8]:
#2.用pandas生成文档词条矩阵
import pandas as pd
from MyDataFrame import MyDataFrame
mdf = MyDataFrame()
df = mdf.new_DataFrame()
df2 = mdf.m_cut(df)

def m_appdf(fileid):#数据框转换函数
    tmpdf = pd.DataFrame(df2['fenci'][fileid],columns=['word'])
    tmpdf['filename'] = df2['filename'][fileid]
    return tmpdf


df0 = pd.DataFrame(columns = ['word','filename'])
for i in range(len(df2)):
    df0 = df0.append(m_appdf(i))#形成一个新的数据集，两列，一列为word，一列为word所属文件的id
    
#1.输出为序列格式
df0.groupby(['word','filename']).agg('size').tail(20)#根据word和filename分组，分组后计算频数，并输出最后10个
    
#2.输出为数据框
t2d = pd.crosstab(df0.word,df0.filename)
t2d.head(100)#行列交叉表

#计算各个词条的总出现频数，准备进行低频减词
totnum = t2d.agg(func='sum',axis = 1)
totnum
#低频词删减
t2dclean = t2d.iloc[list(totnum>=20)]
t2dclean.T

['0', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '4', '5', '6', '7', '8', '9']


word,2.1,2.2,2012,2014,2015,2016,2017,2018,2019,32,...,预测,预测模型,预警,领域,频率,风力,风电,验证,高校,鸡胸
filename,,,,,,,,,,,,,,,,,,,,,
0,1,1,1,1,0,1,1,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,1,0,1,1,2,0,0,...,0,0,0,0,0,0,0,0,0,0
10,1,1,0,3,0,0,1,1,9,0,...,8,0,0,0,1,0,0,0,21,0
11,0,0,0,1,0,0,0,5,4,0,...,7,0,0,1,1,0,0,3,2,0
12,1,1,0,1,0,1,0,0,3,0,...,2,0,1,0,0,0,0,0,0,0
13,1,1,1,0,0,1,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
14,0,0,0,0,0,1,0,1,1,0,...,4,1,1,4,0,0,0,1,0,0
15,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,4,0,0,0,1,0
16,0,0,0,0,1,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0


In [6]:
#3.用sklearns生成文档-词条矩阵
import jieba
from sklearn.feature_extraction.text import CountVectorizer
from MyDataFrame import MyDataFrame
mdf = MyDataFrame()
df = mdf.new_DataFrame()
df2 = mdf.m_cut(df)
rawfile=[]#空格分隔开的文本
for i in range(len(df2)):
    rawfile.append(' '.join(df2['fenci'][i]))
#print(rawfile) 
#rawfile listoflist ,每一项为用空格连接的一篇文档
countvec = CountVectorizer(min_df = 10)#最小词频
res = countvec.fit_transform(rawfile)
res#稀疏矩阵
res.todense()#转置，仍为矩阵
countvec.get_feature_names()#词汇列表，即每列对应的词条

#使用sklearn，读取空格分隔的文本，生成d2m矩阵


type([df.content])

['2009_2016年云南省医疗机构消毒灭菌效果趋势评价_李建云_周晓梅', '_一带一路_背景下_国际投资_课_省略_路径探析_以广西民族师范学院为例_郑国富', '三级医院医疗检查结果互认实施现状及影响因素分析_肖晓华_廖惠_梁恒斌_潘振威_苏', '不同等级医院医疗检查结果互认标准研究_肖晓华_谢欣睿_梁恒斌_廖惠_潘振威_苏茹', '乡村振兴战略下环巢湖休闲农业发展研究_沈东生', '二三级医院影像学检查结果互认现状及影响因素研究_肖晓华_苏茹茹_潘振威_廖惠_梁', '关于建筑工程管理的影响因素分析与对策探讨_王君霞_战丙利', '关于彰武县做好互换并地后续工作的建议_张凤华', '关于深化河北省农业水价综合改革的对策建议_马素英', '农村非正规金融发展与农村多维贫困_基于面板门槛模型的研究_吴君娴_黄永兴', '医师多点执业对医院人力资源管理的影响_张妍_李吉', '协同创新优化现代高效农业组织结构_王瑜', '县级公立医院患者体验的影响因素研究_朱锦_胡丹_陈家应', '双创背景下大学生农业创业融资问题探析_王涛_白林_齐骥霆', '吉林省农村金融发展状况及对策研究_孙铁柱', '国家重点研发计划_畜禽_专项立项特征研究_姜玮', '地方高校工科专业课程教学模式改革_省略_临沂大学_大气污染控制工程_为例_马宏卿', '基于大数据的商业智能在电商数据分析中的应用_钱丹丹', '基于数据挖掘技术的档案馆信息快速分析算法研究_甘璐', '基于数据挖掘的电影票房分析_席稼玮', '基于机器视觉和机器学习技术的鸡胴体质量自动分级方法研究_戚超', '工程教育专业认证下自动化专业实践教学体系的构建_黄宜庆_陆华才', '工程造价指数与工程造价动态管理刍议_孙静', '广西综合医院新生儿病房分级建设和能力建设现状调查_胡琴燕_韦琴_黄晓波_杨少丽_', '应用型高校金融学课程教学改革研究_邓旭霞', '建筑工程施工管理及创新技术的应用研究_侯帅', '建筑工程施工管理的进度管理与控制_吕萌', '建筑结构设计中的抗震结构设计理念_马玉洁', '建筑门窗用硅烷改性聚醚胶的性能与应用研究_杨苏邯_陈洋庆_陈建军_龙飞_蒋金博_', '我国医疗检查结果互认制度实施现状_肖晓华_梁恒斌_谢欣睿_苏茹茹_孙刚', '数据挖掘技术在语音识别中的应用_许元洪_郭琼', '数

list